In [ ]:
# Neo4j GraphRAG Setup
import os
from dotenv import load_dotenv
from neo4j import GraphDatabase
from neo4j_graphrag.llm import OpenAILLM
from neo4j_graphrag.embeddings import OpenAIEmbeddings
from neo4j_graphrag.generation import GraphRAG
from neo4j_graphrag.retrievers import Text2CypherRetriever

# Load environment variables
load_dotenv()

In [ ]:
# Neo4j Connection Setup
NEO4J_URI = os.getenv("NEO4J_URI", "bolt://localhost:7687")
NEO4J_USERNAME = os.getenv("NEO4J_USERNAME", "neo4j")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")

# OpenAI API Configuration
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Initialize Neo4j driver
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

# Initialize LLM and Embeddings
llm = OpenAILLM(
    model_name="gpt-4",
    model_params={"temperature": 0.0},
    openai_api_key=OPENAI_API_KEY
)

embedder = OpenAIEmbeddings(
    model="text-embedding-ada-002",
    openai_api_key=OPENAI_API_KEY
)

In [ ]:
# Initialize GraphRAG
rag = GraphRAG(
    driver=driver,
    llm=llm,
    embedder=embedder
)

# Initialize Text2Cypher Retriever
text2cypher = Text2CypherRetriever(
    driver=driver,
    llm=llm,
    neo4j_schema=None,  # Auto-introspect schema from database
    examples=[
        # Add example queries to help the LLM understand your domain
        "MATCH (n:Person)-[:WORKS_AT]->(c:Company) RETURN n.name, c.name",
        "MATCH (p:Product)-[:BELONGS_TO]->(cat:Category) RETURN p.name, cat.name"
    ]
)

# Test connection
def test_connection():
    try:
        with driver.session() as session:
            result = session.run("RETURN 'Connection successful!' AS message")
            record = result.single()
            print(record["message"])
            return True
    except Exception as e:
        print(f"Connection failed: {e}")
        return False

test_connection()

In [ ]:
# Text2Cypher Query Functions
def text_to_cypher_query(question):
    """Convert natural language to Cypher and execute"""
    try:
        # The Text2CypherRetriever converts natural language to Cypher and executes it
        response = text2cypher.search(query_text=question, top_k=5)
        return response
    except Exception as e:
        print(f"Text2Cypher query failed: {e}")
        return None

# Basic GraphRAG Query (using embeddings)
def basic_rag_query(question):
    """Perform a basic GraphRAG query using embeddings"""
    try:
        response = rag.search(
            query_text=question,
            top_k=5  # Number of relevant results to retrieve
        )
        return response
    except Exception as e:
        print(f"RAG query failed: {e}")
        return None

# Example usage (uncomment when you have data in your graph)
# Text2Cypher examples:
# result1 = text_to_cypher_query("Find all people who work at technology companies")
# result2 = text_to_cypher_query("Show me products in the electronics category")
# result3 = text_to_cypher_query("What are the relationships between nodes?")

# Basic RAG example:
# rag_result = basic_rag_query("What is the main topic discussed in the documents?")

print("Text2Cypher and GraphRAG initialized successfully!")

In [ ]:
# Schema Exploration and Setup
def get_database_schema():
    """Get the current database schema"""
    try:
        with driver.session() as session:
            # Get node labels
            labels_result = session.run("CALL db.labels()")
            labels = [record["label"] for record in labels_result]
            
            # Get relationship types
            rel_types_result = session.run("CALL db.relationshipTypes()")
            relationships = [record["relationshipType"] for record in rel_types_result]
            
            print("Node Labels:", labels)
            print("Relationship Types:", relationships)
            
            return {"labels": labels, "relationships": relationships}
    except Exception as e:
        print(f"Schema retrieval failed: {e}")
        return None

# Update Text2Cypher with current schema (run this after you have data)
def update_text2cypher_schema():
    """Update the Text2Cypher retriever with current database schema"""
    global text2cypher
    try:
        schema_info = get_database_schema()
        if schema_info:
            # Reinitialize with updated examples based on your actual schema
            text2cypher = Text2CypherRetriever(
                driver=driver,
                llm=llm,
                neo4j_schema=None,  # Let it auto-introspect
                examples=[
                    # Update these examples based on your actual data model
                    f"MATCH (n:{schema_info['labels'][0] if schema_info['labels'] else 'Node'}) RETURN n LIMIT 5",
                    "MATCH (n)-[r]->(m) RETURN type(r), labels(n), labels(m) LIMIT 5"
                ]
            )
            print("Text2Cypher updated with current schema")
    except Exception as e:
        print(f"Schema update failed: {e}")

# Uncomment to explore your database schema:
# get_database_schema()